In [1]:
# Skrtas masiniam testavimui - variantas su model_cnn_fda_vu_v1
# Perdarytas iš 6_zive_accuracy_cnn_vu_v2_test_batch.ipynb:
# - pritaikytas variantui, kai klasifikacija izoliuota vienam pūpsniui
# - pakeistas naudojamų požymių sąrašas (pritaikytas vasariniam modeliui 1)
# - naudoja požymių skaičiavimą iš zive_cnn_fda_vu_v2.py (from zive_cnn_fda_vu_v2 import predict_cnn_fda_vu_v1),
# kurį gavau iš Povilo 2022 10 05 - faile ReadSeqEKG_2022_10_05.py: get_spike_width funkcija nesikeičia,
#  keičiasi apply_FDA (perpavardinau į apply_FDA_vasara)
# 
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# Šis variantas pritaikytas npy formato zive įrašams, kuriems pakeistas, lyginant su 
# originaliais įrašais, failo vardas iš `file_name` į `SubjCode`, pridedant `userNr`
# prie `file_name`. 
#
# Skriptas zive EKG pūpsnių CNN VU klasifikatoriaus testavimui ir tikslumo įvertinimui, funkcijos 
# paimamos iš aplanko zive_cnn_fda_vu_v1.py, modelis iš model_cnn_fda_vu_v1, testuojami duomenys
# iš db_folder įrašų saugyklos, jame yra ir failas all_beats_attr. 

# Testavimui imami įrašai iš sąrašo SubjCodes, kuris arba paimamas if failo info_create.json,
# arba iš mokymo, validavimo, testavimo sarašų, pvz. train_subjcode_lst.csv. Visiems įrašams iš šių
# sąrašų egzistuoja informacija apie pūpsnius faile all_beats_attr.
 
# Skripte yra galimybė išvesti ekstrasistolių vietas įraše.
# Dirbant su daug įrašų reiktų užblokuoti: classification = []  # Užblokuota
 
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys, os, json
from pathlib import Path
# from icecream import ic
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support

from zive_util_vu import cm2df, show_confusion_matrix 
from zive_util_vu import create_dir, create_subdir, get_rev_dictionary
from zive_util_vu import runtime, split_SubjCode
from zive_util_vu import get_freq_unique_values

from zive_util_vu import get_beat_attributes
from zive_util_vu import get_userId, read_rec, get_filename 
from zive_util_vu import confusion_matrix_modified, zive_read_df_rpeaks

from zive_cnn_fda_vu_v2 import predict_cnn_fda_vu_v1  # v2 variante pakeistas pakoreguotas požymių
# sąrašas - išmestas santykis Rl/Rr
from zive_cnn_fda_vu_v2 import zive_read_file_1ch
# Pastaba: zive_read_file_1ch importuoju iš zive_cnn_fda_vu_v1, nors ji yra ir zive_util_vu.py
# tam, kad atskirti funkcijas, kurios importuojamos skripte zive analysis, nuo tų funkcijų,
# kurios reikalingos tik zive_accuracy_cnn_vu_v1 ir v2. 

print(tf.__version__)

print("Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui")
print('Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio')

import warnings
warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    # Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
    Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Herkulis
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_VU'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
all_beats_attr_fname = 'all_beats_attr_z.csv'

# Failai pūpsnių klasių formavimui
selected_beats = {'N':0, 'S':1, 'V':2}
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3, 'F':3}  

# Diskretizavimo dažnis:
fs = 200

# /////////////////////////////////////////////////////////////////

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_npy')

# Nuoroda į modelio aplanką
# model_dir = Path(Duomenu_aplankas, 'DNN', 'best_models', 'all_ft')
model_dir = 'model_cnn_fda_vu_v1'

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Zive duomenų aplankas: ", db_folder)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) ", rec_dir)
print("Pūpsnių atributų failas:", all_beats_attr_fname)
print("Diskretizavimo dažnis: ", fs)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print("Modelio ir scaler parametrai nuskaitomas iš aplanko: ", model_dir)

print("\n")


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


2.6.0
Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui
Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio
OS in my system :  linux

Bendras duomenų aplankas:  /home/kesju/DI/Data/MIT&ZIVE/VU
Zive duomenų aplankas:  DUOM_VU
Aplankas su originaliais EKG įrašais ir anotacijomis (.json)  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy
Pūpsnių atributų failas: all_beats_attr_z.csv
Diskretizavimo dažnis:  200
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2}
Klasių skaičius: 3
Modelio ir scaler parametrai nuskaitomas iš aplanko:  model_cnn_fda_vu_v1




In [2]:
# PASIRUOŠIMAS

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

import warnings
# warnings.filterwarnings("ignore")

# Naudojamų požymių sąrašas 
all_features = ['seq_size','RR_l_0', 'RR_r_0', 'RR_r/RR_l','wl_side','wr_side',
                'signal_mean', 'signal_std', 'P_val', 'Q_val', 'R_val', 'S_val', 'T_val',
                'P_pos', 'Q_pos', 'R_pos', 'S_pos', 'T_pos', 'QRS', 'PR', 'ST', 'QT', '0', '1', '2',
                '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
                '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
                '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
                '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
                '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
                '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102',
                '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114',
                '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126',
                '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138',
                '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150',
                '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
                '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174',
                '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186',
                '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198',
                '199']

print("\nAtliekama pūpsnių pacientų įrašuose klasifikacija")

# NURODOME PACIENTŲ SĄRAŠĄ. GALIMI ĮVAIRŪS VARIANTAI

# Variantas: visi duomenys
# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)
# SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas

# Variantas: mokymo imtis  
# file_path = Path(rec_dir, 'train_subjcode_lst.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

# Variantas: validation imtis  
# file_path = Path(rec_dir, 'validation_subjcode_lst.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

# Variantas: testinė imtis  
file_path = Path(rec_dir, 'test_subjcode_lst.csv')
SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

# Pacientų įrašų sąrašas testavimui
# file_path = 'testinis_sarasas.csv'
# SubjCodes = [10020, 10021, 10051] #Testavimui
# SubjCodes = [10010] #Testuojamas pacientų įrašų sąrašas

print("Klasifikuojamų įrašų sąrašas:", SubjCodes)
print(f"Sąrašas nuskaitytas iš: {file_path}")

# Suformuojamas aplankas rezultatams
head_tail = os.path.split(file_path)
filename, file_extension = os.path.splitext(head_tail[1])
path_for_results = Path(db_path, 'rezultatai' + '_' + filename)
print(f"\nAplankas rezultatams: {path_for_results}")
create_dir(path_for_results)

# Kas kiek išvedamas apdorotų sekų skaičius
show_period = 100

# Klasių simbolinių vardų sąrašas ir klasių skaičius
class_names = list(selected_beats.keys()) 
n_classes = len(selected_beats)
# print(class_names)

# Nuskaitome pūpsnių atributų masyvą
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr = pd.read_csv(file_path, index_col=0, dtype = {'userNr': int, 'recordingNr': int,
                                                             'sample': int, 'symbol': str, 'label': int})
all_beat_indices = all_beats_attr.index

index_start = 0
# Sukūriame masyvą, į kurį sudėsime visų įrašų pūpsnių anotuotus ir automatiškai surastus klasių numerius
validation_set_stats = pd.DataFrame(columns=['idx', 'test_label', 'pred_label', 'SubjCode'])

start_time = time.time()
# Ciklas per pacientų įrašus
for SubjCode in SubjCodes:
    
    # Nuskaitome EKG įrašą (npy formatu)
    sign_raw = read_rec(rec_dir, SubjCode)
    signal_length = sign_raw.shape[0]
    signal = sign_raw

    # Surandame ir išvedame įrašo atributus
    file_name = get_filename(rec_dir, SubjCode)
    userNr, recNr = split_SubjCode(SubjCode)
    print(f"\nSubjCode: {SubjCode} userNr: {userNr:>2} file_name: {file_name:>2} signal_length: {signal_length}")

    # Filtruojame signalą
    # signal = signal_filter(signal=sign_raw, sampling_rate=200, lowcut=0.2, method="butterworth", order=5)

    # Nuskaitome paciento anotacijas ir jų indeksus
    df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()

    # SUFORMUOJAME EKG ĮRAŠUI TESTINĮ ir PRISKIRTŲ KLASIŲ NUMERIŲ MASYVUS

    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])

    (unique, counts) = np.unique(test_labels, return_counts=True)
    total = counts.sum()
    print("test_labels: ", unique, counts, total)
   
    pred_labels = predict_cnn_fda_vu_v1(signal, atr_sample, model_dir)
    # pred_labels turi būti tokio pat ilgio, kaip ir test_labels, praleisti (šiuo atveju pirmas
    # ir paskutinis pūpsnys), o taip pat pakliuvęs į ommited sritį, pažymimi klase 3
    if (len(test_labels) != len(pred_labels)):
        raise Exception(f"Klaida! SubjCode: {SubjCode}. Nesutampa test_labels ir pred_labels ilgiai")     
    
    # print("test_labels:", len(test_labels))
    # print(test_labels)
    # print("pred_labels:", len(pred_labels))
    # print(pred_labels)
    
    (unique, counts) = np.unique(pred_labels, return_counts=True)
    total = counts.sum()
    print("pred_labels: ",unique, counts, total)

    # Surandame vietas su ekstrasistolemis ir išvedame jų sąrašą vizualiniam įvertinimui. 
    classification=[]
    for i, i_sample in enumerate(atr_sample):
        if ((pred_labels[i] != 0) or test_labels[i] != 0):
            classification.append({'sample':i_sample, 'annot':test_labels[i], 'pred':pred_labels[i]})

    # Vietų sąrašas išvedamas
    # Dirbant su daug įrašų sąrašo išvedimą reikia užblokuoti !!!!!!!!!!!!!!!!!!!!!!!!!!!
    classification = []  # Užblokuota
    if (classification):
        for row in classification:
            print(f"sample: {row['sample']:>7}   annot_label: {row['annot']:>2}   pred_label: {row['pred']:>2}")  
    
    # SUFORMUOJAME FREIMĄ validation_set_stats SU PŪPSNIŲ KLASIŲ ANOTUOTAIS IR AUTOMATIŠKAI 
    # SURASTAIS KLASIŲ NUMERIAIS, IŠMETANT KLASES SU NUMERIU = 3

    # Surandame pradinį SubjCode įrašo indeksą faile all_beats_attr
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recNr)]
    # print(f"SubjCode: {SubjCode}  first elem: {selected_ind[0]} last elem: {selected_ind[-1]}  tot: {len(selected_ind)}")
    index_start = selected_ind[0]
    # print('\nSubjCode:',SubjCode, 'index_start:', index_start)   

    #  Praleisdami indeksą, jei masyvuose test_labels ir pred_labels yra reikšmė == 3,
    # suformuojame klasifikuotinų pūpsnių indeksų sąrašą
    for idx in range(len(atr_sample)):
        flag = (test_labels[idx] == 3) or (pred_labels[idx] == 3)
        if (flag == False):
            # Dėmesio: ?????????????????????????????????????????????
            # taisytina vieta, bus problemų su pandas 1.4.1
            validation_set_stats = validation_set_stats.append({'idx':index_start+idx, 
            'test_label':test_labels[idx],'pred_label':pred_labels[idx], 'SubjCode': SubjCode}, ignore_index=True)

    # Suformuojame klasių numerių msyvus confusion matricai skaičiuoti, surandama confusion matrica
    test_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['test_label']).astype('int') 
    # print(all_beats_attr.info())
    pred_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['pred_label']).astype('int')
   
    # Atsikračius pūpsnių su klase = 3 ir suformavus masyvus, pred_y turi būti tokio pat ilgio, kaip ir test_y
    if (len(test_y) != len(pred_y)):
        raise Exception(f"Klaida! SubjCode: {SubjCode}. Nesutampa test_y ir pred_y ilgiai")     

    confusion = confusion_matrix(test_y, pred_y)
    # print(confusion)
    prec,rec,fsc,sup = precision_recall_fscore_support(test_y, pred_y, labels=[0, 1, 2], zero_division=0)

    str1 =f"N:{int(sup[0]):>5} S:{(int(sup[1])):3} V:{int(sup[2]):3}" 
    str2 = f"  Nprec:{prec[0]:>5.2f} Nrec:{rec[0]:5.2f} Nfsc:{fsc[0]:5.2f}"
    str3 = f"  Sprec:{prec[1]:>5.2f} Srec:{rec[1]:5.2f} Sfsc:{fsc[1]:5.2f}"
    str4 = f"  Vprec:{prec[2]:>5.2f} Vrec:{rec[2]:5.2f} Vfsc:{fsc[2]:5.2f}"
    print(str1+str2+str3+str4)

    # print(len(validation_set_stats))
    # print(len(test_y))
    # print(len(pred_y))

end_time = time.time()
print('\n')
runtime(end_time-start_time)

# Sukūriame anotuotų ir automatiškai priskirtų klasių visų įrašų pūpsniams sąrašus 
validate_ind_lst = list(validation_set_stats['idx'])
y_validate = np.array(validation_set_stats['test_label']).astype('int')
y_predicted = np.array(validation_set_stats['pred_label']).astype('int')



Atliekama pūpsnių pacientų įrašuose klasifikacija
Klasifikuojamų įrašų sąrašas: [10000, 10001, 10002, 10040, 10041, 10042, 10120, 10121, 10122, 10190, 10191, 10192, 10193, 10194, 10195, 10196, 10197, 10198, 10199, 10200, 10201, 10202, 10260, 10261, 10262, 10263, 10264, 10265, 10266, 10267, 10268, 10269, 10300, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10320, 10420, 10421, 10422, 10423, 10424, 10425, 10426, 10427, 10428, 10429, 10450, 10470, 10520, 10521, 10522, 10523, 10524, 10525, 10526, 10527, 10528, 10529, 10540, 10541, 10542, 10543, 10544, 10545, 10546, 10547, 10548, 10549, 10600, 10601, 10602, 10710, 10711, 10712, 10713, 10714, 10715, 10716, 10717, 10718, 10719, 10720, 10721, 10722, 10723, 10724, 10725, 10726, 10727, 10728, 10729, 10770, 10771, 10772, 10773, 10774, 10775, 10776, 10777, 10778, 10779, 10820]
Sąrašas nuskaitytas iš: /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy/test_subjcode_lst.csv

Aplankas rezultatams: /home/kesju/DI/Data/MIT&ZIVE/VU/DU

2022-10-10 10:01:55.825313: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-10 10:01:55.825653: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-10-10 10:02:04.423421: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


pred_labels:  [0 1 3] [759   4   3] 766
N:  759 S:  4 V:  0  Nprec: 1.00 Nrec: 1.00 Nfsc: 1.00  Sprec: 1.00 Srec: 1.00 Sfsc: 1.00  Vprec: 0.00 Vrec: 0.00 Vfsc: 0.00

SubjCode: 10001 userNr: 1000 file_name: 1626941.468 signal_length: 127999
test_labels:  [0 1] [859   2] 861
pred_labels:  [0 1 3] [857   2   2] 861
N:  857 S:  2 V:  0  Nprec: 1.00 Nrec: 1.00 Nfsc: 1.00  Sprec: 1.00 Srec: 1.00 Sfsc: 1.00  Vprec: 0.00 Vrec: 0.00 Vfsc: 0.00

SubjCode: 10002 userNr: 1000 file_name: 1632923.661 signal_length: 575999
test_labels:  [0 1] [4849    1] 4850
pred_labels:  [0 1 3] [4838    9    3] 4850
N: 4846 S:  1 V:  0  Nprec: 1.00 Nrec: 1.00 Nfsc: 1.00  Sprec: 0.00 Srec: 0.00 Sfsc: 0.00  Vprec: 0.00 Vrec: 0.00 Vfsc: 0.00

SubjCode: 10040 userNr: 1004 file_name: 1630715.197 signal_length: 127999
test_labels:  [0 1] [610   2] 612
pred_labels:  [0 3] [610   2] 612
N:  608 S:  2 V:  0  Nprec: 1.00 Nrec: 1.00 Nfsc: 1.00  Sprec: 0.00 Srec: 0.00 Sfsc: 0.00  Vprec: 0.00 Vrec: 0.00 Vfsc: 0.00

SubjCode: 1

In [3]:
# MODELIO TIKSLUMO VERTINIMO IŠ VERTINIMO IMTIES REZULTATAI

print("\nMODELIO TIKSLUMO VERTINIMO REZULTATAI")
print("Modelis iš aplanko: ", model_dir)

cols, dist, tot = get_freq_unique_values(y_validate, ['N', 'S', 'V'])
print(f"Klasės {cols}: {dist} Suma: {tot} ")

# APIBENDRINTI REZULTATAI

print('\nAPIBENDRINTI REZULTATAI\n')

# Skaičiuojame ir išvedame klasifikavimo lentelę
confusion = confusion_matrix(y_validate, y_predicted)
pd.set_option('display.precision',3)
show_confusion_matrix(confusion, class_names)
# print('\n')

print("\nClassification Report\n")
# target_names = [key for (key, value) in selected_beats.items()]

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

print(classification_report(y_validate, y_predicted, target_names=class_names, digits=3))
report = classification_report(y_validate, y_predicted, target_names=class_names, output_dict=True)
# output_dictbool, default=False, If True, return output as dict.
df_report = pd.DataFrame(report).transpose()
# https://medium.com/@asmaiya/you-can-something-like-this-84d28e0fd31f

# Įrašome į diską
filepath = Path(path_for_results, 'apibendrinti_rezultatai.csv') 
df_report.to_csv(filepath)    
print(f'\nApibendrinti_rezultatai įrašyti į:  {filepath}')



MODELIO TIKSLUMO VERTINIMO REZULTATAI
Modelis iš aplanko:  model_cnn_fda_vu_v1
Klasės ['N', 'S', 'V']: [78968  1279  2550] Suma: 82797 

APIBENDRINTI REZULTATAI

Confusion Matrix
       N    S     V
N  78749  149    70
S    486  611   182
V    139   38  2373


Normalized Confusion Matrix
      N     S     V
N 0.997 0.002 0.001
S 0.380 0.478 0.142
V 0.055 0.015 0.931

Classification Report

              precision    recall  f1-score   support

           N      0.992     0.997     0.995     78968
           S      0.766     0.478     0.588      1279
           V      0.904     0.931     0.917      2550

    accuracy                          0.987     82797
   macro avg      0.887     0.802     0.833     82797
weighted avg      0.986     0.987     0.986     82797


Apibendrinti_rezultatai įrašyti į:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/rezultatai_test_subjcode_lst/apibendrinti_rezultatai.csv


In [6]:
# KLAIDŲ PASISKIRSTYMAS PER PACIENTUS IR JŲ ĮRAŠUS

from zive_util_vu import zive_read_df_data, create_SubjCode
from sklearn.metrics import accuracy_score

def collect_noise_locs(rec_dir, all_beats_attr):

    noise_arr_tot = np.empty(0, dtype=int)
    
    grouped = all_beats_attr.groupby(['userNr', 'recordingNr'])
    for key, group in grouped:
        # print('\n',key)
        userNr = key[0]
        recordingNr = key[1]

        # SubjCode naudojamas atveju, kai duomenis yra formoje SubjCode.npy, SubjCode.json
        SubjCode = create_SubjCode(userNr, recordingNr)
        filepath = Path(rec_dir, str(SubjCode) + '.json') 
        df_noises = zive_read_df_data(filepath, 'noises')

        noise_arr = np.full(shape=len(group), fill_value=0,  dtype=int)
        idx_noise = 0

        for i, row_i in group.iterrows():
            sample = row_i['sample']
            for j, row_j in df_noises.iterrows():
                if ((sample > row_j['startIndex']) & (sample < row_j['endIndex'])):
                    noise_arr[idx_noise] = 1
                    idx_noise += 1
        noise_arr_tot = np.append(noise_arr_tot, noise_arr)
    return noise_arr_tot

def get_error(y_test, y_pred):
    # Error in %
    n_errors = 0
    for idx in range(len(y_pred)):
        if (y_test[idx] != y_pred[idx]):
            n_errors += 1
    Err = float(n_errors)/len(y_pred)*100.
    Err = round(Err, 1)
    return Err

# Pasiruošimas

pd.set_option("display.max_rows", 6000, "display.max_columns", 15)
pd.set_option('display.width', 1000)

# Sukuriame ir užpildome dataframe su sekų parametrais
df_seq_errors = pd.DataFrame(columns= ['idx', 'userNr', 'file_name'])

rows_list = []
for idx in validate_ind_lst:
# Surandame  userNr, recordingNr, symbol
    userNr, recNr, label, symbol = get_beat_attributes(idx, all_beats_attr)
    seq_attr = {'idx': idx, 'userNr':userNr, 'recordingNr':recNr}
    rows_list.append(seq_attr)

# print(rows_list[:10])

# Čia įdedame informaciją, ar pūpsnys patenka į triukšmų zoną
noise_arr = collect_noise_locs(rec_dir, all_beats_attr)
noise_arr = noise_arr[validate_ind_lst]

# print(noise_arr)
# unique, counts = np.unique(noise_arr, return_counts=True)
# print(unique, counts, noise_arr.shape[0])
# print(counts.sum())

df_seq_errors = pd.DataFrame(rows_list)

df_seq_errors['labels'] = pd.Series(y_validate)
df_seq_errors['preds'] = pd.Series(y_predicted)
zeros_arr = np.zeros( y_predicted.shape[0], dtype=int)
df_seq_errors['errors'] = pd.Series(zeros_arr)  
df_seq_errors.loc[df_seq_errors['labels'] != df_seq_errors['preds'], 'errors'] = 1 
df_seq_errors['noises'] = pd.Series(noise_arr)  

# print(df_seq_errors.info())


# Rezultatų pasiskirstymas per pacientus

print("\nRezultatų pasiskirstymas per pacientus")

# Pasiruošimas
class_names = ['N', 'S', 'V']
n_classes = len(class_names)
df_user_errors = pd.DataFrame({'userNr':pd.Series(dtype='int'), 'userId':pd.Series(dtype='str'),  # ??????????????????????
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
    'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# Išgauname pacientų vidinę (eilės nr) numeraciją
grouped  = df_seq_errors.groupby(['userNr'])
userNrs = list(grouped.groups.keys())
print(f'Pacientų: {len(userNrs)}\n')

for userNr in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin

    y_test = df_seq_errors.loc[grouped.groups[userNr]]['labels'].to_numpy(dtype=int)
    y_pred = df_seq_errors.loc[grouped.groups[userNr]]['preds'].to_numpy(dtype=int)

    Err = get_error(y_test, y_pred)
    
    noise_arr = df_seq_errors.loc[grouped.groups[userNr]]['noises'].to_numpy(dtype=int)
    Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.

    # Testavimui
    # acc = accuracy_score(y_test, y_pred)
    # print(f"userNr: {userNr} recordingNr:  {recordingNr} Accuracy: {acc:.2f}")
    # cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
    # show_confusion_matrix(cnf_matrix, class_names)
    # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826

    prec,rec,fsc,sup = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1, 2], zero_division=0)
    userId = get_userId(rec_dir, userNr)

    dict_user_errors = {'userNr':int(userNr),'userId':str(userId),   # //////////////////////////////////////////
    'N':sup[0], 'S':sup[1], 'V':sup[2],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err,  'Noise%':Noise
    }
    rows_list.append(dict_user_errors)

df_user_errors =  pd.DataFrame(rows_list) 

# Išvedame suformuotą masyvą
tit1 = f"{'userNr':>6} {'userId':>24} {'N':>8} {'S':>4} {'V':>4}"
tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
print(tit1+tit2+tit3+tit4)

for idx, row in  df_user_errors.iterrows():
    str1 =f"{int(row['userNr']):>6} {str(row['userId']):>6} {int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
    str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
    str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
    str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
    print(str1+str2+str3+str4)

filepath = Path(path_for_results, 'rezultatu_pasiskirstymas_per_pacientus.csv') 
df_user_errors.to_csv(filepath)    
print(f'\nRezultatų pasiskirstymas per pacientus įrašytas į:  {filepath}')

# print(df_user_errors)




Rezultatų pasiskirstymas per pacientus
Pacientų: 2

userNr                   userId        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
  1002 60e1d80f93b55b41529e9eaa     1900    6   29    0.99  0.99  0.99    0.14  0.33  0.20    0.78  0.24  0.37      2.0     63.7
  1005 613b1c6f3d08d4370acdc8f3      536    4    0    0.99  1.00  1.00    0.00  0.00  0.00    0.00  0.00  0.00      0.7      0.0

Rezultatų pasiskirstymas per pacientus įrašytas į:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\rezultatai_testinis_sarasas\rezultatu_pasiskirstymas_per_pacientus.csv


In [7]:
# Rezultatų pasiskirstymas per pacientus ir jų įrašus
# Skaičiuojama visoms 3 klasėms Precision(tikslumas), Recall (atgaminimas), Fscore (F rodiklis)
# 
# https://towardsdatascience.com/you-dont-always-have-to-loop-through-rows-in-pandas-22a970b347ac
# 
pd.set_option("display.max_rows", 6000, "display.max_columns", 18)
pd.set_option('display.width', 1000)

# Rezultatų pasiskirstymas per pacientų įrašus
print("\n\nRezultatų pasiskirstymas per pacientų įrašus")

# Pasiruošimas
class_names = ['N', 'S', 'V']
n_classes = len(class_names)
df_rec_errors = pd.DataFrame({'userNr':pd.Series(dtype='int'),
    'recordingNr':pd.Series(dtype='int'), 'file_name':pd.Series(dtype='str'),  
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
     'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# Sugrupuojame eilutes pagal 'userId','recordingId', suskaičiuojame, kiek kiekviename įraše
# iš viso yra klasifikuojamų sekų (labels) ir kiek padaryta klaidų (errors).
# Grupavimo objektą paverčiame į normalų dataframe objektą

grouped  = df_seq_errors.groupby(['userNr','recordingNr'])
print(f'Pacientų įrašų: {grouped.ngroups}')
# print(f'{grouped.size()=}')

for key in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin

    # print(f'\nGroup: {key}\n{df_seq_errors.loc[grouped.groups[key]]}')
    userNr = key[0]
    recordingNr = key[1]
    file_name = get_filename(rec_dir, create_SubjCode(userNr, recordingNr))
    userId = get_userId(rec_dir, userNr)

    y_test = df_seq_errors.loc[grouped.groups[key]]['labels'].to_numpy(dtype=int)
    y_pred = df_seq_errors.loc[grouped.groups[key]]['preds'].to_numpy(dtype=int)

    Err = get_error(y_test, y_pred)

    noise_arr = df_seq_errors.loc[grouped.groups[key]]['noises'].to_numpy(dtype=int)
    Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.


    # *********************** Testavimui *******************************************************************
    # acc = accuracy_score(y_test, y_pred)
    # print(f"userNr: {userNr} recordingNr:  {recordingNr} Accuracy: {acc:.2f}")
    # cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
    # show_confusion_matrix(cnf_matrix, class_names)
    # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826
    # *********************************************************************************************************

    prec,rec,fsc,sup = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1, 2], zero_division=0)
    
    dict_rec_errors = {'userNr':int(userNr), 'recordingNr':recordingNr, 'file_name':file_name,
    'N':sup[0], 'S':sup[1], 'V':sup[2],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err, 'Noise%':Noise
    }
    rows_list.append(dict_rec_errors)

df_rec_errors =  pd.DataFrame(rows_list) 

# Išvedame suformuotą masyvą
grouped  = df_rec_errors.groupby('userNr')
for userNr, group in grouped:
    # print(group.dtypes)
    print("\n")
    userId = get_userId(rec_dir, userNr)
    print(f"{'userNr:'} {userNr} {'userId:'} {userId}" )
    tit1 = f"{'recordingNr':>6} {'file_name':>15} {'N':>8} {'S':>4} {'V':>4}"
    tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
    tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
    tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
    print(tit1+tit2+tit3+tit4)

    for idx, row in group.iterrows():
        str1 =f"{int(row['recordingNr']):>6} {str(row['file_name']):>20} {int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
        str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
        str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
        str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
        print(str1+str2+str3+str4)

filepath = Path(path_for_results, 'rezultatu_pasiskirstymas_per_irasus.csv') 
df_rec_errors.to_csv(filepath)    
print(f'\nRezultatų pasiskirstymas per įrašus įrašytas į:  {filepath}')




Rezultatų pasiskirstymas per pacientų įrašus
Pacientų įrašų: 3


userNr: 1002 userId: 60e1d80f93b55b41529e9eaa
recordingNr       file_name        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
     0          1625408.182     1156    2   17    0.99  0.99  0.99    0.08  0.50  0.14    0.75  0.18  0.29      2.2     68.6
     1          1625402.027      744    4   12    0.99  1.00  0.99    0.50  0.25  0.33    0.80  0.33  0.47      1.7     56.2


userNr: 1005 userId: 613b1c6f3d08d4370acdc8f3
recordingNr       file_name        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
     1          1630735.143      536    4    0    0.99  1.00  1.00    0.00  0.00  0.00    0.00  0.00  0.00      0.7      0.0

Rezultatų pasiskirstymas per įrašus įrašytas į:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\rezultatai_testinis_sarasas\rezultatu_pasiskirstymas_per_irasus.csv
